In [36]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip list

Package                   Version
------------------------- --------------
aiohappyeyeballs          2.4.3
aiohttp                   3.10.10
aiosignal                 1.3.1
annotated-types           0.7.0
anyio                     4.6.2.post1
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.4.0
colorama                  0.4.6
comm                      0.2.2
dataclasses-json          0.6.7
debugpy                   1.8.7
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.1.0
fastjsonschema            2.20.0
fqdn                      1.5.1
frozenlist                1.4.1
greenlet                  3.1.1
h11       

In [2]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv



In [10]:
from langchain.chat_models import ChatOpenAI 

In [12]:
pip install json_repair

Note: you may need to restart the kernel to use updated packages.


In [13]:
import json_repair

In [19]:
load_dotenv()

True

In [4]:
graph = Neo4jGraph()

In [6]:
loader = TextLoader(file_path="dummytext.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [14]:
llm_type = os.getenv("LLM_TYPE", "ollama")

# Vérifie si le type de LLM est 'ollama'
if llm_type == "ollama":
    llm = ChatOllama(model="llama3.1", temperature=0)
else:
    # Si ce n'est pas 'ollama', utilise OpenAI GPT-4 à la place
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# Crée un transformateur LLM pour convertir les documents
llm_transformer = LLMGraphTransformer(llm=llm)

# Convertit les documents en graphes avec le transformateur
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [15]:
graph_documents[0]

GraphDocument(nodes=[Node(id='The Story of Amico’s Family: A Legacy of Love and Tradition', type='Text', properties={}), Node(id="Amico's Family", type='Family', properties={})], relationships=[Relationship(source=Node(id="Amico's Family", type='Family', properties={}), target=Node(id='The Story of Amico’s Family: A Legacy of Love and Tradition', type='Text', properties={}), type='IS_DESCRIBED_IN', properties={})], source=Document(metadata={'source': 'dummytext.txt'}, page_content='1. The Story of Amicoâ€™s Family: A Legacy of Love and Tradition'))

In [16]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [92]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [20]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

In [21]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

NotImplementedError: 

In [95]:
entity_chain.invoke("Who are Nonna Lucia and Giovanni Caruso?")

Entities(names=['Nonna Lucia', 'Giovanni Caruso'])

In [96]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [97]:
print(graph_retriever("Who is Nonna Lucia?"))

Nonna Lucia - INFLUENCED -> Caponata
Nonna Lucia - INFLUENCED -> Fresh Pasta
Nonna Lucia - TAUGHT -> Amico
Lucia - USES -> Organic And Local Ingredients
Lucia - PRACTICES -> Sustainable Cooking Practices
Bella Vita - OWNER -> Lucia


In [99]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [100]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [101]:
chain.invoke(input="Who is Nonna Lucia? Did she teach anyone about restaurants or cooking?")

"Nonna Lucia is the matriarch of the Caruso family, a culinary sage who taught her grandchildren, including Amico, the art of Sicilian cooking. She's also the owner of Bella Vita."